In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [2]:
# checkpoint_dir = "/pub/hofmann-scratch/dvruette/gidd-easydel/gidd-unif-3b/gidd/orbax"
# checkpoint_dir = "/pub/hofmann-scratch/dvruette/gidd-easydel/gidd-mask-3b/gidd/orbax"
checkpoint_dir = "/pub/hofmann-scratch/dvruette/gidd-easydel/gidd-unif-10b/gidd/orbax"

In [3]:
state_dict_path = os.path.join(checkpoint_dir, "..", "state_dict.pt")
pretrained_path = os.path.join(checkpoint_dir, "..", "pretrained")

if "gidd-unif-3b" in checkpoint_dir:
    num_layers = 19
    hidden_size = 3072
    num_attn_heads = 24
    noise_type = 1000.0
elif "gidd-mask-3b" in checkpoint_dir:
    num_layers = 18
    hidden_size = 3072
    num_attn_heads = 24
    noise_type = -1000.0
elif "gidd-unif-10b" in checkpoint_dir:
    num_layers = 34
    hidden_size = 4608
    num_attn_heads = 36
    noise_type = 1000.0
else:
    raise ValueError("Unknown checkpoint")

In [4]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("dvruette/nemotron-cc-bpe")

In [5]:
import torch

import importlib
import gidd_easydel.model.modeling_gidd_hf
import gidd_easydel.model.configuration_gidd
importlib.reload(gidd_easydel.model.modeling_gidd_hf)
importlib.reload(gidd_easydel.model.configuration_gidd)

from gidd_easydel.model.modeling_gidd_hf import GiddForDiffusionLM
from gidd_easydel.model.configuration_gidd import GiddConfig

device = torch.device("cuda")
# device = torch.device("cpu")

config = GiddConfig(
    num_hidden_layers=num_layers,
    hidden_size=hidden_size,
    num_attention_heads=num_attn_heads,
    intermediate_size=hidden_size * 4,
    max_position_embeddings=2048,
    resid_scale=4.0,
    init_scale = 0.4 / hidden_size ** 0.5,
    weight_scaling=1.0,
    head_scaling=512.0 / hidden_size,
    use_qk_norm=True,
    attention_bias=True,
    mlp_bias=True,
    torch_dtype=torch.bfloat16,
    auto_map={
        "AutoConfig": "configuration_gidd.GiddConfig",
        "AutoModel": "modeling_gidd.GiddModel",
        "AutoModelForCausalLM": "modeling_gidd.GiddForDiffusionLM",
    },
    attn_performer="eager",
    noise_type=noise_type,
)

model = GiddForDiffusionLM(config)

HELLO FROM EASYDEL


2025-12-14 20:32:02,480 - eray-executor - INFO - Not initializing jax.distributed because no distributed config was provided, and no cluster was detected.
2025-12-14 20:32:02,481 - eray-executor - INFO - No auto-discovered ray address found. Using ray.init('local').
2025-12-14 20:32:02,481 - eray-executor - INFO - ray.init(address='local', namespace='eray-executor', **{})
2025-12-14 20:32:05,771	INFO worker.py:1908 -- Started a local Ray instance. View the dashboard at http://127.0.0.1:8265 


HELLO FROM ORBAX


INFO:2025-12-14 20:32:11,390:jax._src.xla_bridge:822: Unable to initialize backend 'tpu': INTERNAL: Failed to open libtpu.so: libtpu.so: cannot open shared object file: No such file or directory
2025-12-14 20:32:11,390 - jax._src.xla_bridge - INFO - Unable to initialize backend 'tpu': INTERNAL: Failed to open libtpu.so: libtpu.so: cannot open shared object file: No such file or directory


In [6]:
state_dict = torch.load(state_dict_path, map_location="cpu")
model.load_state_dict(state_dict)

model.eval()
torch.cuda.empty_cache()
model.to(device);

In [7]:
# generated_ids = model.generate(
#     inputs=torch.tensor([[tokenizer.bos_token_id]], device=device, dtype=torch.long),
#     max_length=128,
#     steps=256,
#     block_length=128,
#     sampling_method="ancestral",
#     noise_schedule="cosine",
#     show_progress=True,
# )

In [8]:
# tokenizer.batch_decode(generated_ids, skip_special_tokens=False)

In [7]:
prompts = [
    """Question: Jen and Tyler are gymnasts practicing flips. Jen is practicing the triple-flip while Tyler is practicing the double-flip. Jen did sixteen triple-flips during practice. Tyler flipped in the air half the number of times Jen did. How many double-flips did Tyler do?
Answer: Jen did 16 triple-flips, so she did 16 * 3 = <<16*3=48>>48 flips.
Tyler did half the number of flips, so he did 48 / 2 = <<48/2=24>>24 flips.
A double flip has two flips, so Tyler did 24 / 2 = <<24/2=12>>12 double-flips.
#### 12

Question: Four people in a law firm are planning a party. Mary will buy a platter of pasta for $20 and a loaf of bread for $2. Elle and Andrea will split the cost for buying 4 cans of soda which cost $1.50 each, and chicken wings for $10. Joe will buy a cake that costs $5. How much more will Mary spend than the rest of the firm put together?
Answer: Mary will spend $20 + $2 = $<<20+2=22>>22.
Elle and Andrea will spend $1.5 x 4 = $<<1.5*4=6>>6 for the soda.
Elle and Andrea will spend $6 + $10 = $<<6+10=16>>16 for the soda and chicken wings.
Elle, Andrea, and Joe together will spend $16 + $5 = $<<16+5=21>>21.
So, Mary will spend $22 - $21 = $<<22-21=1>>1 more than all of them combined.
#### 1

Question: A charcoal grill burns fifteen coals to ash every twenty minutes of grilling. The grill ran for long enough to burn three bags of coals. Each bag of coal contains 60 coals. How long did the grill run?
Answer: The grill burned 3 * 60 = <<3*60=180>>180 coals.
It takes 20 minutes to burn 15 coals, so the grill ran for 180 / 15 * 20 = <<180/15*20=240>>240 minutes.
#### 240

Question: A bear is preparing to hibernate for the winter and needs to gain 1000 pounds. At the end of summer, the bear feasts on berries and small woodland animals. During autumn, it devours acorns and salmon. It gained a fifth of the weight it needed from berries during summer, and during autumn, it gained twice that amount from acorns. Salmon made up half of the remaining weight it had needed to gain. How many pounds did it gain eating small animals?
Answer: The bear gained 1 / 5 * 1000 = <<1/5*1000=200>>200 pounds from berries.
It gained 2 * 200 = <<2*200=400>>400 pounds from acorns.
It still needed 1000 - 200 - 400 = <<1000-200-400=400>>400 pounds.
Thus, it gained 400 / 2 = <<400/2=200>>200 pounds from salmon.
Therefore, the bear gained 400 - 200 = <<400-200=200>>200 pounds from small animals.
#### 200

Question: Brendan can cut 8 yards of grass per day, he bought a lawnmower and it helped him to cut more yards by Fifty percent per day. How many yards will Brendan be able to cut after a week?
Answer: The additional yard Brendan can cut after buying the lawnmower is 8 x 0.50 = <<8*0.50=4>>4 yards.
So, the total yards he can cut with the lawnmower is 8 + 4 = <<8+4=12>>12.
Therefore, the total number of yards he can cut in a week is 12 x 7 = <<12*7=84>>84 yards.
#### 84

Question: Janet’s ducks lay 16 eggs per day. She eats three for breakfast every morning and bakes muffins for her friends every day with four. She sells the remainder at the farmers' market daily for $2 per fresh duck egg. How much in dollars does she make every day at the farmers' market?
Answer:"""
]

prompt_ids = tokenizer(prompts, add_special_tokens=True, return_tensors="pt").input_ids[:, :-1]

In [8]:
generated_ids = model.generate(
    inputs=prompt_ids,
    max_length=128,
    block_length=128,
    steps=256,
    sampling_method="adaptive",
    noise_schedule="linear",
    temperature=0.0,
    tokens_per_step=1,
    show_progress=True,
)

  0%|          | 0/256 [00:00<?, ?it/s]

In [9]:
print(tokenizer.batch_decode(generated_ids, skip_special_tokens=False)[0])

<|begin_of_text|> Question : Jen and Tyler are gymnasts practicing flips . Jen is practicing the triple -flip while Tyler is practicing the double -flip . Jen did sixteen triple -flips during practice . Tyler flipped in the air half the number of times Jen did . How many double -flips did Tyler do ?
 Answer : Jen did  16 triple -flips , so she did  16 *  3 = << 16 * 3 = 48 >> 48 flips .
 Tyler did half the number of flips , so he did  48 /  2 = << 48 / 2 = 24 >> 24 flips .
 A double flip has two flips , so Tyler did  24 /  2 = << 24 / 2 = 12 >> 12 double -flips .
 ####  12 

 Question : Four people in a law firm are planning a party . Mary will buy a platter of pasta for $ 20 and a loaf of bread for $ 2 . Elle and Andrea will split the cost for buying  4 cans of soda which cost $ 1 . 50 each , and chicken wings for $ 10 . Joe will buy a cake that costs $ 5 . How much more will Mary spend than the rest of the firm put together ?
 Answer : Mary will spend $ 20 + $ 2 = $<< 20 + 2 = 22 >> 

In [10]:
import shutil

shutil.rmtree(pretrained_path, ignore_errors=True)
model.save_pretrained(pretrained_path, max_shard_size="5GB")
tokenizer.save_pretrained(pretrained_path)
shutil.copyfile("./gidd_easydel/model/configuration_gidd.py", os.path.join(pretrained_path, "configuration_gidd.py"))
shutil.copyfile("./gidd_easydel/model/modeling_gidd_hf.py", os.path.join(pretrained_path, "modeling_gidd.py"))

'/pub/hofmann-scratch/dvruette/gidd-easydel/gidd-unif-10b/gidd/orbax/../pretrained/modeling_gidd.py'

In [8]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoConfig

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

tokenizer = AutoTokenizer.from_pretrained(pretrained_path, trust_remote_code=True)
config = AutoConfig.from_pretrained(pretrained_path, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(pretrained_path, trust_remote_code=True).to(device)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [15]:
torch.set_float32_matmul_precision("high")

input_ids = torch.arange(8, device=device).unsqueeze(0)
noise_mask = torch.ones((1, 8), dtype=torch.bool, device=device)
noise_mask[:, :noise_mask.shape[1] // 2] = False

attn_mask = attn_mask = (noise_mask[..., None] >= noise_mask[..., None, :])

with torch.no_grad():
    outputs = model(
        input_ids=input_ids,
        attention_mask=attn_mask,
        output_hidden_states=True,
    )

torch.save(tuple(map(lambda x: x.cpu(), outputs.hidden_states)), "hidden_states.pt")
torch.save(outputs.logits.cpu(), "logits.pt")

In [ ]:
outputs.logits[0, [0, 1, 2, 3, 4, 5, 6, 7], [0, 1, 2, 3, 4, 5, 6, 7]]

tensor([-8.7500,  4.0312, -8.3125,  0.0000, 11.8125, 10.3125,  8.0000,  3.5625],
       dtype=torch.bfloat16)

In [ ]:
outputs.hidden_states[1][0, 1, 222]

tensor(0., dtype=torch.bfloat16)

In [ ]:
outputs.hidden_states[-1]

tensor([[[-0.0095,  0.0038, -0.0270,  ..., -0.0009, -0.0100,  0.0481],
         [-0.0275,  0.0483, -0.0479,  ..., -0.0439, -0.0289, -0.0728],
         [ 0.0161,  0.0815,  0.0408,  ...,  0.0072,  0.0040,  0.0515],
         ...,
         [-0.0029, -0.0007, -0.0273,  ...,  0.0415,  0.0046,  0.0101],
         [-0.0391,  0.0640,  0.0334,  ..., -0.0161, -0.0231,  0.0728],
         [ 0.0115, -0.0154, -0.0540,  ..., -0.0055, -0.0151,  0.0728]]],
       dtype=torch.bfloat16)

In [ ]:
outputs.logits

tensor([[[-8.6875,  1.2266, -8.4375,  ..., -6.1562, -3.9844, -3.0625],
         [-8.3125,  3.9531, -9.0000,  ..., -6.8438, -4.0938, -5.7812],
         [-8.0000,  5.9688, -8.1875,  ..., -4.7188, -5.6562, -1.3438],
         ...,
         [-6.9688,  7.7188, -7.9062,  ..., -6.0312, -9.8125, -4.0938],
         [-4.7812,  9.1250, -6.2812,  ..., -4.0625, -6.6250, -4.2188],
         [-5.5000,  7.2812, -6.3438,  ..., -4.3438, -7.9062, -2.3438]]],
       dtype=torch.bfloat16)

In [ ]:
len(outputs.hidden_states)

20